In [33]:
import pandas as pd
import requests
import pickle

In [5]:
#Read list of URLs to sent to Google API into dataframe
df = pd.read_excel('Google API urls - Python.xlsx',header=None)
df.columns = ['urls']

In [19]:
df.shape

(1728, 1)

In [18]:
df.head()

,urls
0,https://maps.googleapis.com/maps/api/distancem...
1,https://maps.googleapis.com/maps/api/distancem...
2,https://maps.googleapis.com/maps/api/distancem...
3,https://maps.googleapis.com/maps/api/distancem...
4,https://maps.googleapis.com/maps/api/distancem...


In [21]:
#Send each URL to API and capture response in new column in dataframe
df['response'] = df['urls'].map(lambda x: requests.get(x).json())

In [24]:
df.tail()

,urls,response
1723,https://maps.googleapis.com/maps/api/distancem...,{'destination_addresses': ['3710-3714 Slopevie...
1724,https://maps.googleapis.com/maps/api/distancem...,{'destination_addresses': ['3710-3714 Slopevie...
1725,https://maps.googleapis.com/maps/api/distancem...,{'destination_addresses': ['3710-3714 Slopevie...
1726,https://maps.googleapis.com/maps/api/distancem...,{'destination_addresses': ['3710-3714 Slopevie...
1727,https://maps.googleapis.com/maps/api/distancem...,{'destination_addresses': ['3710-3714 Slopevie...


In [36]:
#Save responses into pickle file so don't have to make API calls again if something goes wrong later
with open('dataframe.p', 'wb') as f:
    pickle.dump(df,f)

In [53]:
#Define function to unpack the json response into individual fields
def unpack_json(data):
    if data['status'] == 'OK':
        values = data['rows'][0]['elements'][0]
        return [
            data['destination_addresses'][0],
            data['origin_addresses'][0],
            values['distance']['value'],
            values['duration']['value'],
            values.get('duration_in_traffic', {'value':None})['value']
        ]
    else:
        return [''] * 5

In [54]:
#Run function and store data into new columns in dataframe
df['dest'], df['orgin'], df['dist'], df['dura'], df['dura_t'] = zip(*df['response'].map(unpack_json))

In [55]:
df.head()

,urls,response,dest,orgin,dist,dura,dura_t
0,https://maps.googleapis.com/maps/api/distancem...,"{'destination_addresses': ['552-598 Broadway, ...","552-598 Broadway, Redwood City, CA 94063, USA","Junipero Serra Fwy, Los Altos Hills, CA 94022,...",19148,1098,923
1,https://maps.googleapis.com/maps/api/distancem...,"{'destination_addresses': ['552-598 Broadway, ...","552-598 Broadway, Redwood City, CA 94063, USA","Junipero Serra Fwy, Los Altos Hills, CA 94022,...",19148,1098,988
2,https://maps.googleapis.com/maps/api/distancem...,"{'destination_addresses': ['552-598 Broadway, ...","552-598 Broadway, Redwood City, CA 94063, USA","Junipero Serra Fwy, Los Altos Hills, CA 94022,...",19148,1098,1110
3,https://maps.googleapis.com/maps/api/distancem...,"{'destination_addresses': ['552-598 Broadway, ...","552-598 Broadway, Redwood City, CA 94063, USA","Junipero Serra Fwy, Los Altos Hills, CA 94022,...",19148,1098,1290
4,https://maps.googleapis.com/maps/api/distancem...,"{'destination_addresses': ['552-598 Broadway, ...","552-598 Broadway, Redwood City, CA 94063, USA","Junipero Serra Fwy, Los Altos Hills, CA 94022,...",19148,1098,1175


In [56]:
#Write final dataframe to csv file
df.to_csv('results.csv', index=False)